In [2]:
import gzip
import json

def messages_parse(data):
    data_with_no_binary = str(data).split("'")[1:-1][0]
    data_list =[i for i in data_with_no_binary.split('\\n') if i != '']
    messages_data = [json.loads(D) for D in data_list]
    return messages_data 

def messages_read(file="../data/coinbase_BTC-USD_20_10_06_000000-010000.json.gz"):
    file_object = gzip.open(file, "r")
    data = file_object.read()
    messages_data = messages_parse(data)
    return messages_data 


def snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json"):
    snaphsot_data = json.loads(open(file,'r').read())
    return snaphsot_data

messages_data = messages_read()
initial_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_00.json")
final_clob = snapshot_read(file="../data/coinbase_BTC-USD_20_10_06_00_15.json")

In [8]:
def messages_filter(messages_data,initial_clob,final_clob):
    messages_data_filtered = [message_dict for message_dict in messages_data if message_dict['sequence'] >= initial_clob['sequence'] and message_dict['sequence'] <= final_clob['sequence']]
    return messages_data_filtered
                                                                                        
messages_data_filtered = messages_filter(messages_data,initial_clob,final_clob)

                                                                                        
                                                                                    

In [9]:
print (len(messages_data))
print (len(messages_data_filtered))

1086672
252618


In [4]:
print (messages_data[0])
print (initial_clob.keys())
print (final_clob.keys())

{'type': 'done', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:00.021717Z', 'sequence': 16712803594, 'order_id': '0e612005-8b3c-4b34-a2db-eadc20afc95e', 'reason': 'canceled', 'price': '10802.46', 'remaining_size': '0.25'}
dict_keys(['bids', 'asks', 'sequence'])
dict_keys(['bids', 'asks', 'sequence'])


In [28]:
print (final_clob['bids'][:100])

[['10792.35', '3.1', '642537c6-d66f-4bcc-8a00-6ee2ad422b27'], ['10791.06', '0.5', '4b90fe46-315a-4e58-9e2c-87221275e6f2'], ['10791.03', '3.1', '15222e4c-59dd-479a-bc53-8c83ff1567ca'], ['10790.82', '0.501', '975388db-5dbd-4dbe-9d40-b4949902cecd'], ['10790.35', '0.01345822', '21dd930a-e507-4a27-b30e-9b947dd6bb8c'], ['10790.18', '0.1829', 'd5180194-e2a8-4413-8914-76c46f461c3d'], ['10790.07', '0.2', 'df21a9d1-bf62-42f5-8b75-b25d0a85aaa4'], ['10789.62', '0.27620805', '81f030eb-ab66-466f-8d3c-2466671a55f7'], ['10789.61', '3.1', '94b394b7-62dd-4510-9fee-7f2a081da10d'], ['10789.6', '3.1', '45971a7b-7921-4a60-840e-e14c8d1d7066'], ['10789.32', '0.5', '882b598a-1aac-405b-88d2-a652d6393e85'], ['10789.18', '3.1', 'c5fd4f7c-d1f9-4848-aafe-54712e0371cf'], ['10788.94', '3.1', 'db0737ad-8b00-4224-a806-25129ea0ac9f'], ['10788.93', '2.80522372', '4c53de3f-b8e4-4538-9efb-11d324615719'], ['10788.8', '3.08320458', '52c9f4ec-9a87-4676-beb1-cc628e578fb9'], ['10788.8', '0.1', 'ef5ae8fb-ddd6-4ea2-8684-05508bafc

In [27]:
messages_data_filtered[100]

{'type': 'open',
 'side': 'buy',
 'product_id': 'BTC-USD',
 'time': '2020-10-06T00:00:05.860994Z',
 'sequence': 16712805003,
 'price': '10778.91',
 'order_id': '5cee9745-3cfa-46ee-b48d-31511a163ae7',
 'remaining_size': '1.31839767'}

In [36]:

filter_order_id_message = lambda array,order_id: [D for D in array if 'order_id' in D.keys() and D['order_id'] ==order_id]
filter_order_id_message(messages_data_filtered,"19b9d70e-8821-4846-ba72-ce5eeba2feaa")


[]

In [23]:
filter_order_id = lambda clob,asks_or_bids,order_id: [subli for subli in clob[asks_or_bids] if subli[2] == order_id]
filter_order_id(final_clob,'asks','d98e69bd-aa7c-40ec-8c34-5087c20b86bd')

[]

In [16]:
len([message_dict for message_dict in messages_data_filtered if 'order_id' not in message_dict.keys()])

1077

In [29]:
import pandas as pd

# Orders in Messages

In [32]:
def order_ids_pull(orders_data_dict,asks_or_bids):
    return [D[2] for D in orders_data_dict[asks_or_bids]]

def messages_identify(messages_data,order_ids):
    return [D for D in messages_data if 'order_id' in D.keys() and D['order_id'] in order_ids]

order_ids = order_ids_pull(final_clob,'asks')
messages_with_orders = messages_identify(messages_data_filtered,order_ids)
print (len(messages_with_orders))

358


In [33]:
print (messages_with_orders[0])

{'type': 'received', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:06.075997Z', 'sequence': 16712805032, 'order_id': '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6', 'order_type': 'limit', 'size': '0.001', 'price': '10800.05', 'client_oid': ''}


In [35]:
filter_order_id = lambda clob,asks_or_bids,order_id: [subli for subli in clob[asks_or_bids] if subli[2] == order_id]

filter_order_id(final_clob,'asks','9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6')

[['10800.05', '0.001', '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6']]

In [37]:
filter_order_id_message(messages_data_filtered,'9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6')


[{'type': 'received',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.075997Z',
  'sequence': 16712805032,
  'order_id': '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6',
  'order_type': 'limit',
  'size': '0.001',
  'price': '10800.05',
  'client_oid': ''},
 {'type': 'open',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.075997Z',
  'sequence': 16712805033,
  'price': '10800.05',
  'order_id': '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6',
  'remaining_size': '0.001'}]

In [38]:
filter_order_id_message(messages_data,'9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6')


[{'type': 'received',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.075997Z',
  'sequence': 16712805032,
  'order_id': '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6',
  'order_type': 'limit',
  'size': '0.001',
  'price': '10800.05',
  'client_oid': ''},
 {'type': 'open',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:06.075997Z',
  'sequence': 16712805033,
  'price': '10800.05',
  'order_id': '9ee2c3ef-0309-406d-a0a0-c0e1599cd9b6',
  'remaining_size': '0.001'}]

In [40]:
print (messages_with_orders[2])

{'type': 'received', 'side': 'sell', 'product_id': 'BTC-USD', 'time': '2020-10-06T00:00:30.894232Z', 'sequence': 16712811974, 'order_id': '6f4dbd88-e19a-4fec-aec4-efd433acb663', 'order_type': 'limit', 'size': '0.1', 'price': '10850', 'client_oid': '31a1208f-1f79-4b6d-85b2-a0cdb2a435f7'}


In [42]:
filter_order_id_message(messages_data_filtered,'6f4dbd88-e19a-4fec-aec4-efd433acb663')


[{'type': 'received',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:30.894232Z',
  'sequence': 16712811974,
  'order_id': '6f4dbd88-e19a-4fec-aec4-efd433acb663',
  'order_type': 'limit',
  'size': '0.1',
  'price': '10850',
  'client_oid': '31a1208f-1f79-4b6d-85b2-a0cdb2a435f7'},
 {'type': 'open',
  'side': 'sell',
  'product_id': 'BTC-USD',
  'time': '2020-10-06T00:00:30.894232Z',
  'sequence': 16712811975,
  'price': '10850',
  'order_id': '6f4dbd88-e19a-4fec-aec4-efd433acb663',
  'remaining_size': '0.1'}]

In [43]:
filter_order_id(final_clob,'asks','6f4dbd88-e19a-4fec-aec4-efd433acb663')

[['10850', '0.1', '6f4dbd88-e19a-4fec-aec4-efd433acb663']]

In [44]:
len(final_clob['asks'])

12677

In [45]:
len(final_clob['bids'])

28413

## Assess Change Between Initial and Final Snapshot to find specific differences
What are the differences between initial snapshot and final snapshot

## Look at Match Messages to See if Change
How do match messages change the order book?

## Open Messages are not in order book

## What happens if I group OrderID Messages together?